In [2]:
import time

import lingua

# Getting Started

There is a bit of documentation on how to interact with the large models [here](https://lingua-sdk.readthedocs.io/en/latest/getting_started.html). The relevant github links to the SDK are [here](https://github.com/VectorInstitute/lingua-sdk) and underlying code [here](https://github.com/VectorInstitute/lingua).

First we connect to the service through which, we'll interact with the LLMs and see which models are avaiable to us

In [3]:
# Establish a client connection to the Lingua service
client = lingua.Client(gateway_host="llm.cluster.local", gateway_port=3001)

Show all supported models

In [4]:
client.models

['OPT-175B', 'OPT-6.7B']

Show all model instances that are currently active

In [5]:
client.model_instances

[{'id': 'c402a90b-5867-476b-950d-9921585335ec',
  'name': 'OPT-6.7B',
  'state': 'ACTIVE'},
 {'id': 'af334811-a4fc-483d-91be-a65a3a98d34e',
  'name': 'OPT-175B',
  'state': 'ACTIVE'}]

Let's start by querying the OPT-175B model. We'll try other models below. Get a handle to a model. In this example, let's use the OPT-175B model.

In [6]:
model = client.load_model("OPT-6.7B")
# If this model is not actively running, it will get launched in the background.
# In this case, wait until it moves into an "ACTIVE" state before proceeding.
while model.state != "ACTIVE":
    time.sleep(1)

We need to configure the model to generate in the way we want it to. We set important parameters.

*`max_tokens` sets the number the model generates before haulting generation.
*`top_k`: Range: 0-Vocab size. At each generation step this is the number of tokens to select from with relative probabilities associated with their likliehoods. Setting this to 1 is "Greedy decoding." If top_k is set to zero them we exclusively use nucleus sample (i.e. top_p below).
*`top_p`: Range: 0.0-1.0, nucleus sampling. At each generation step, the tokens the largest probabilities, adding up to `top_p` are sampled from relative to their likliehoods.
*`rep_penalty`: Range >= 1.0. This attempts to decrease the likelihood of tokens in a generation process if they have been generated before. A value of 1.0 means no penalty and larger values increasingly penalize repeated values. 1.2 has been reported as a good default value.
*`temperature`: Range >=0.0. This value "sharpens" or flattens the softmax calculation done to produce probabilties over the vocab. As temperature goes to zero: only the largest probabilities will remain non-zero (approaches greedy decoding). As it approaches infinity, the distribution spreads out evenly over the vocabulary.

In [7]:
long_generation_config = {"max_tokens": 128, "top_k": 4, "top_p": 3, "rep_penalty": 1.2, "temperature": 0.5}
short_generation_config = {"max_tokens": 10, "top_k": 4, "top_p": 3, "rep_penalty": 1.2, "temperature": 0.5}
single_word_generation_config = {"max_tokens": 1, "top_k": 4, "top_p": 3, "rep_penalty": 1.2, "temperature": 0.5}

Let's try a basic prompt for factual information.

__Note__ that if you run the cell multiple times, you'll get different responses due to sampling.

In [8]:
generation = model.generate("What is the capital of Canada?", long_generation_config)
# Extract the text from the returned generation
generation.generation["text"]

["\nOttawa.\nNope.   Ottawa is the capital of Canada, but Toronto is the capital of Canada's largest city.\nThat's what I meant, sorry."]

Now let's create a basic prompt template that we can reuse for multiple text inputs. This will be an instruction prompt with an unconstrained answer space. We'll try several different templates and examine performance for each.

In [9]:
prompt_template_summary_1 = "Summarize the preceding text."
prompt_template_summary_2 = "Short Summary:"
prompt_template_summary_3 = "TLDR;"

In [10]:
with open("resources/examples_news.txt", "r") as file:
    news_stories = file.readlines()

In [11]:
prompts_with_template_1 = [f"{news_story} {prompt_template_summary_1}" for news_story in news_stories]
prompts_with_template_2 = [f"{news_story} {prompt_template_summary_2}" for news_story in news_stories]
prompts_with_template_3 = [f"{news_story} {prompt_template_summary_3}" for news_story in news_stories]

In [12]:
generation_1 = model.generate(prompts_with_template_1, long_generation_config)
for summary, original_story in zip(generation_1.generation["text"], news_stories):
    print(f"Original Length: {len(original_story)}, Summary Length: {len(summary)}")
    print(summary)
    print("====================================================================================")
    print("")

Original Length: 1262, Summary Length: 557


In recent years, Iran has received a number of weapons and equipment from Russia, including the S-300, S-400, Russian-made air defense systems, and Iskander missiles. Iran has used these weapons in its support of the Assad regime in Syria.

On the other hand, Iran has also received weapons and equipment from the United States and NATO.

In particular, the United States has been supplying Ukraine with Javelins, Stinger anti-aircraft systems, and other weapons and equipment.

The United States has also provided Ukraine with TOW anti-tank weapons.

The

Original Length: 1181, Summary Length: 639
 How does the information relate to the topic of this report?

The National Weather Service has issued a flash flood watch for the San Francisco Bay Area and the Central Coast, as well as a flood advisory for the Santa Cruz Mountains and the Santa Lucia Mountains.

What is the most important information for readers to know about this report?

The Nation

In [13]:
generation_2 = model.generate(prompts_with_template_2, long_generation_config)
for summary, original_story in zip(generation_2.generation["text"], news_stories):
    print(f"Original Length: {len(original_story)}, Summary Length: {len(summary)}")
    print(summary)
    print("====================================================================================")
    print("")

Original Length: 1262, Summary Length: 579
 Kiev says it is working to prevent the delivery of Russian-made weapons to Iran, but some of the weapons appear to have made it there.

Ukraine is working to prevent the delivery of Russian-made weapons to Iran, but some of the weapons appear to have made it there.

The US and its allies are worried that Iran could reverse-engineer the weapons and use them against US forces in the Middle East.

The US and its allies are worried that Iran could reverse-engineer the weapons and use them against US forces in the Middle East.

Russia has been capturing some of the US and NATO

Original Length: 1181, Summary Length: 680


Residents in parts of Ventura and Santa Barbara counties in California have been issued evacuation warnings in anticipation of a powerful storm that is expected to deliver heavy rain and cause widespread flooding across the central and northern parts of the state.

The most dangerous amount of rain could impact nearly 70,000 peop

In [14]:
generation_3 = model.generate(prompts_with_template_3, long_generation_config)
for summary, original_story in zip(generation_3.generation["text"], news_stories):
    print(f"Original Length: {len(original_story)}, Summary Length: {len(summary)}")
    print(summary)
    print("====================================================================================")
    print("")

Original Length: 1262, Summary Length: 503
 Russia is taking captured US and NATO weapons systems and sending them to Iran to reverse engineer them.

I don't think the US and NATO are going to be too happy about that.

__________________

Quote:

Originally Posted by SolidGold

Bortlezzzzzzz

Quote:

Originally Posted by Monomach

Brilliant letting one of Scott Pioli's henchmen have his own team to ruin. One of the premier GM jobs in the NFL and it gets handed to a stupid **** who makes three facepalm moves for every good one. Awesome. Just

Original Length: 1181, Summary Length: 660

California is bracing for another round of heavy rainfall and flooding, as the state remains in recovery mode from prior storms.
The most dangerous amount of rain could impact nearly 70,000 people along the central California coast, stretching from Salinas southward to San Luis Obispo and including parts of Ventura and Monterey counties, according to the Weather Prediction Center, which issued a Level 4 o

Story 1 is about the possibility of severe flooding in California and an evacuation order being issued. Let's see if we get that from the three summaries and maybe which worked better.

In [15]:
print(f"{prompt_template_summary_1}|| {generation_1.generation['text'][1]}")
print("====================================================================================")
print(f"{prompt_template_summary_2}|| {generation_2.generation['text'][1]}")
print("====================================================================================")
print(f"{prompt_template_summary_3}|| {generation_3.generation['text'][1]}")

Summarize the preceding text.||  How does the information relate to the topic of this report?

The National Weather Service has issued a flash flood watch for the San Francisco Bay Area and the Central Coast, as well as a flood advisory for the Santa Cruz Mountains and the Santa Lucia Mountains.

What is the most important information for readers to know about this report?

The National Weather Service has issued a flash flood watch for the San Francisco Bay Area and the Central Coast, as well as a flood advisory for the Santa Cruz Mountains and the Santa Lucia Mountains.

What are the possible impacts?

The National Weather Service has issued a flash flood watch
Short Summary:|| 

Residents in parts of Ventura and Santa Barbara counties in California have been issued evacuation warnings in anticipation of a powerful storm that is expected to deliver heavy rain and cause widespread flooding across the central and northern parts of the state.

The most dangerous amount of rain could imp

Can we improve the results by providing additional instructions and a little post-processing?

In [26]:
prompt_template_summary_4 = "Summarize the text in as few words as possible:"
prompts_with_template_4 = [f"{news_story} {prompt_template_summary_4}" for news_story in news_stories]
generation_4 = model.generate(prompts_with_template_4, long_generation_config)
for summary, original_story in zip(generation_4.generation["text"], news_stories):
    print(summary.strip().split(".")[0])
    print("====================================================================================")
    print("")

The article is a summary of a longer article

Officials in California issued evacuation warnings in portions of several counties amid powerful storms likely to deliver severe rainfall and cause widespread flooding across the central and northern parts of the state Friday

West Virginia is asking the Supreme Court to allow it to enforce a state law that prohibits transgender women and girls from participating in public school sports



OPT and generative models in general have been reported to perform better when not prompted with "declarative" instructions. Let's ask it as a question!

In [25]:
prompt_template_summary_5 = "How would you briefly summarize the text?"
prompts_with_template_5 = [f"{news_story} {prompt_template_summary_5}" for news_story in news_stories]
generation_5 = model.generate(prompts_with_template_5, long_generation_config)
for summary, original_story in zip(generation_5.generation["text"], news_stories):
    print(summary.strip().split("\n")[0])
    print("====================================================================================")
    print("")

The article reports that the Ukrainian military has been reporting losses of US equipment to Russia, and that the US believes that Russia is reverse-engineering the equipment.

The article is about a severe storm that is expected to impact the central and northern parts of California. It is also about the impact of the storm on the state, including the flooding in January.

The law prohibits trans women and girls from participating in sports in public schools, colleges and universities. The law says “a person is a male or female based on the gender assigned to the individual at birth.” The law says “a person is a male or female based on the gender assigned to the individual at birth.” It also says “a person is a male or female based on the gender assigned to the individual at birth.” The law says “a person is a male or female based on the gender assigned to the individual at birth.” It also says “a person is a



Finally, let's ask what this story is about

In [30]:
prompt_template_summary_6 = "Briefly, what is this story about?"
prompts_with_template_6 = [f"{news_story} {prompt_template_summary_6}" for news_story in news_stories]
generation_6 = model.generate(prompts_with_template_6, long_generation_config)
for summary, original_story in zip(generation_6.generation["text"], news_stories):
    print(summary.strip().split("\n")[0])
    print("====================================================================================")
    print("")

The story is about the US arming of Ukraine. The US has been arming Ukraine since the beginning of the war.

The National Weather Service issued an excessive rainfall warning for the Salinas, Santa Maria, Santa Barbara and Ventura areas.

The law in question, House Bill 3293, was passed in 2021, and signed by Justice in April. The bill prohibits transgender athletes from participating in girls’ and women’s sports. The law applies to high school and middle school sports. It also requires that athletes who are assigned male at birth to compete in boys’ sports, and vice versa. The law does not apply to college or professional sports, nor does it apply to interscholastic sports. The law also requires that an athlete’s “biological sex” be listed on their birth certificate. The law does not require athletes to disclose



In [28]:
prompt_template_summary_7 = "In short,"
prompts_with_template_7 = [f"{news_story} {prompt_template_summary_7}" for news_story in news_stories]
generation_7 = model.generate(prompts_with_template_7, long_generation_config)
for summary, original_story in zip(generation_7.generation["text"], news_stories):
    print(summary.strip().split("\n")[0])
    print("====================================================================================")
    print("")

the US is aware that Russia is taking advantage of the situation and is attempting to use it to its advantage.

California is in the middle of a very bad storm.

the state’s lawyers argue that the 4th Circuit’s ruling prevents the state from “enforcing the West Virginia law and defending the law against future litigation.” The ACLU, which is representing the student athlete, called the state’s request “meritless.” “The 4th Circuit’s decision is based on the merits of the case and should not be disturbed,” the ACLU said in a statement. “The law violates the Equal Protection Clause and the Due Process Clause of the Fourteenth Amendment, and it does not advance any legitimate state interest.”

